# Exercise 3: Finite size scaling

In this exercise, we will determine the critical temperature and critical exponents of the
Ising model.

a) First, we need finite size data. Since generating the data takes some time, it is useful
to save it to disk. Generate your own finite size data from last weeks program (which
can take some time to get well converged results), and/or download finite size data
provided on the GitLab page. Inspect generate_data.py to find out how the data
is structured and how to load the data. Plot the specific heat CV and magnetic
susceptibility χ.

Recall: 
$$ C_v = \frac{1}{k_BT^2} \left(\left<E^2\right> - \left<E\right>^2\right)  $$

$$ \chi = \frac{1}{k_BT} \left(\left<M^2\right> - \left<M\right>^2\right) $$


In [ ]:
%matplotlib qt5
from ising_model import IsingModel
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
def calc_Cv(E: np.ndarray, T: float, kb: float = 1) -> float:
    return 1/(kb*T**2) * np.var(E)

calc_Cv_vectorized = np.vectorize(calc_Cv, excluded=["E"], signature="(m),()->()")

def calc_chi(M: np.ndarray, kbT: float) -> float:
    return 1/(kbT) * np.var(np.abs(M))

calc_chi_vectorized = np.vectorize(calc_chi, excluded=["M"], signature="(m),()->()")


In [ ]:
import os
if not os.path.exists("ising_data"):
    os.mkdir("ising_data")
    
def get_data(L: int, T0: float = 1, Tn: float = 3.5, N: int = 100) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    filename = f"ising_data/{L=}.npz"
    try:
        file = np.load(filename)
        return file["Es"], file["Ms"], file["Ts"]
    
    except OSError:
        system = IsingModel(1.0, L, L)
        Ts = np.linspace(T0, Tn, N)

        # do some thermalization
        for _ in range(100):
            system.iterate_swendsen_wang(Ts[0])

        Es, Ms = system.sweep_swendsen_wang(Ts, 200)

        np.savez(filename, Es=Es, Ms=Ms, Ts=Ts)

        return Es, Ms, Ts

In [ ]:
Es_20, Ms_20, Ts = get_data(20)
# Es_100, Ms_100 = generate_data(100)

In [ ]:
Cvs_20 = calc_Cv_vectorized(Es_20, Ts)
chis_20 = calc_chi_vectorized(Ms_20, Ts)
# Cvs_100 = calc_Cv_vectorized(Es_100, Ts)
# chis_100 = calc_chi_vectorized(Ms_100, Ts)

In [ ]:
plt.figure()
plt.suptitle("Verification plot")
plt.subplot(1,2,1)
plt.plot(Ts, Cvs_20 / 20**2, label="L=20")
# plt.plot(Ts, Cvs_100 / 100**2, label="L=100")
plt.xlabel("T")
plt.legend()
plt.title("Cv")
plt.subplot(1,2,2)
plt.plot(Ts, chis_20, label="L=20")
# plt.plot(Ts, chis_100, label="L=100")
plt.xlabel("T")
plt.title("$\\chi$")
plt.legend()
plt.show()

b) The specific heat CV and magnetic susceptibility χ have maxima, which move with
increasing system size L. Determine the temperature corresponding to the maxima
for various system sizes and plot them versus $\frac{1}{L}$. Extrapolate to L → ∞ to obtain
an estimate for the critical temperature Tc.
Hint: You can use the functions `np.argmax` and `np.polyfit`.

In [ ]:
Ls = np.array([5, 10, 15, 20, 40, 60])
Tcs_Cv = []
Tcs_chi = []
for L in Ls:
    Es, Ms, Ts = get_data(L)
    Cvs = calc_Cv_vectorized(Es, Ts)
    chis = calc_chi_vectorized(Ms, Ts)

    Tc_Cv = Ts[np.argmax(Cvs)]
    Tc_chi = Ts[np.argmax(chis)]

    Tcs_Cv.append(Tcs_Cv)
    Tcs_chi.append(Tcs_chi)

plt.figure()
plt.subplot(1,2,1)
plt.plot(1/Ls, Tcs_Cv)
plt.title("Tc from Cv")
plt.subplot(1,2,2)
plt.plot(1/Ls, Tcs_chi)
plt.title("Tc from $\\chi$")
plt.show()